# Test 5 (paper) - Experimento 5 (PT)

Entrenamiento de Mask R-CNN con dataset modificado para emular "Test 5"

### Hiperparametros
* **epoch = 120**
    * steps x epoch = 146 (lotes de imagenes)
    * batch = 2
* optimizador = SGD
* Funcion de perdida = SMOOTHL1LOSS
* Metrica de evaluacion = mAP (IoU >= 0.5)
* Mini-mask shape: 56x56
* RPN anchor scales: (32, 64, 128, 256, 512)
* Tasa de aprendizaje: 0.001
* imagenes = 292
    * entrenamiento 81% = 237
    * validacion 19% = 55
* etiquetas = 9140
* **resolucion = 1920 x 1080**
* etiquetas = bounding box formato VOC XML
* **numero de clases = 2 (arandano, arandano-maduro)**
* **data augmentation = false**


## Comprobar directorio principal

In [1]:
!pwd && ls -l

/tf/PT_JoseVeloso/Mask_RCNN-master_matterport/model-training
total 1688
drwxr-xr-x 4 root root   4096 Aug  7 06:10 build
drwxr-xr-x 2 root root   4096 Aug  7 06:10 dist
drwxr-xr-x 2 root root   4096 Aug  8 03:15 mask_rcnn.egg-info
-rw-r--r-- 1 root root  67789 Sep 16 23:55 master-test_11.ipynb
-rw-r--r-- 1 root root  88232 Sep 17 07:27 master-test_11_fase_2-Copy1.ipynb
-rw-r--r-- 1 root root  67792 Sep 17 08:36 master-test_11_fase_2.ipynb
-rw-r--r-- 1 root root  30144 Sep 16 23:44 master-test_12.ipynb
-rw-r--r-- 1 root root  30144 Sep 16 22:02 master-test_12_fase_2.ipynb
-rw-r--r-- 1 root root 350337 Sep 15 00:38 master-test_5-fase_2.ipynb
-rw-r--r-- 1 root root 350144 Sep 17 08:42 master-test_5_fase_2.ipynb
-rw-r--r-- 1 root root 111017 Sep 18 16:23 master-test_6.ipynb
-rw-r--r-- 1 root root 270148 Sep 17 16:09 master-test_6_fase_2.ipynb
drwxr-xr-x 4 root root   4096 Sep 18 19:33 mrcnn
drwxr-xr-x 3 root root   4096 Sep 15 02:27 old
-rw-r--r-- 1 root root 163924 Sep 18 23:12 test_5_cor

# Importar bibliotecas

In [2]:
# bibliotecas basicas
import os
from os import listdir
import sys
import json
import datetime

#sys.path.append("/tf/PT_JoseVeloso/Mask_RCNN-master/")

# bibliotecas avanzadas 
from xml.etree import ElementTree
import skimage.draw
import cv2
import imgaug

# bibliotecas mask rcnn 
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from mrcnn import visualize

# biblioteca matplotlib 
import matplotlib.pyplot as plt

# bibliotecas numpy 
import numpy as np
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from numpy import mean

# bibliotecas keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img   #keras.preprocessing.image tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

# ignorar alertas de elementos que seran descontinuados
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

%matplotlib inline
#plt.show()

import imgaug.augmenters as iaa

# Fase 2 - Entrenamiento con dos clases y etiquetas de Bounding Box

Entrenamiento del modelo con 2 clases:
arandano, arandano-maduro
Las imágenes estan etiquetadas con bounding box de la misma manera que en Fase 1.

In [3]:
cd /tf/PT_JoseVeloso/Mask_RCNN-master_matterport/

/tf/PT_JoseVeloso/Mask_RCNN-master_matterport


In [4]:
class DatasetArandanos(Dataset):
    
    # la funcion load_dataset es usada para cargar el dataset de entrenamiento y test
    def load_dataset(self, dataset_dir, is_train=True):
        
        # se agrega una clase que se necesita para clasificar, en este caso arandano
        # self.add_class('source', 'class id', 'class name')
        self.add_class("dataset", 1, "arandano")
        self.add_class("dataset", 2, "arandano-maduro")
        #self.add_class("dataset", 3, "arandano-pinton")
        #self.add_class("dataset", 4, "arandano-semi")
        
        # se concatena dataset_dir con /images y /annots
        images_dir = dataset_dir + '/images/'
        annotations_dir = dataset_dir + '/annots/'
        
        # is_train sera Verdadero si se esta entrenando el modelo y Falso si se esta testeando el modelo
        for filename in listdir(images_dir):
            
            # extract image id
            image_id = filename[:-4] # se usa para omitir los últimos 4 caracteres: '.jpg' (en class_id.jpg)
            
            # si is_train es Verdadero se omiten todas las imágenes con id mayor que e iguales a 11074
            # aproximadamente el 80% del conjunto de datos es para entrenamiento
            if is_train and int(image_id) >= 41590 :
                #print("image_id: ", image_id)
                continue
             
            # si is_train no es Verdadero se omiten todas las imágenes con id menores a 11074
            if not is_train and int(image_id) < 41590:
                continue
            
            # se declara la ruta de la imagen y la ruta de las etiquetas 
            img_path = images_dir + filename
            ann_path = annotations_dir + image_id + '.xml'
            
            # usando la función add_image se pasan image_id, image_path y ann_path para que la 
            # imagen actual se agregue al conjunto de datos para entrenamiento o prueba
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    # funcino usada para extraer bouding boxes desde archivos etiquetados 
    def extract_boxes(self, filename):

        # se puede ver en las imágenes que estan etiquetadas, como se extraen los valores de ancho, alto y bndbox

        # <size>

        #       <width>640</width>

        #       <height>360</height>

        #       <depth>3</depth>

        # </size>


        # <object>

        #          <name>damage</name>

        #          <pose>Unspecified</pose>

        #          <truncated>0</truncated>

        #          <difficult>0</difficult>


        #          <bndbox>

        #                 <xmin>315</xmin>

        #                 <ymin>160</ymin>

        #                 <xmax>381</xmax>

        #                 <ymax>199</ymax>

        #          </bndbox>

        # </object>

        # </annotation>
        
        # para analizar los archivos .xml
        tree = ElementTree.parse(filename)
        
        # para obtener la raíz del archivo xml
        root = tree.getroot()
        
        # se agregan todas las coordenadas x, y en boxes para todas las instancias de un objeto
        boxes = list()
        
        # se encuentran todos los atributos con el nombre bndbox que existan para cada ground truth en la imagen
        for box in root.findall('.//object'):
            name = box.find('name').text
            xmin = int(box.find('./bndbox/xmin').text)
            ymin = int(box.find('./bndbox/ymin').text)
            xmax = int(box.find('./bndbox/xmax').text)
            ymax = int(box.find('./bndbox/ymax').text)
            coors = [xmin, ymin, xmax, ymax, name]
            boxes.append(coors)
                        
            #quita todas las imagenes no etiquetadas
            if name=='arandano' or name=='arandano-maduro':
                boxes.append(coors)

        # extraer ancho y alto de la imagen
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        
        # retorna boxes-> list, width-> int y height-> int 
        return boxes, width, height
    
    # esta función llama al método extract_boxes y se usa para cargar una máscara para cada instancia en una imagen 
    # devuelve una máscara booleana con las siguientes dimensiones ancho * alto * instancias
    def load_mask(self, image_id):
        
        # info apunta al image_id actual 
        info = self.image_info[image_id]
        
        # se obtiene la ruta de anotación de image_id que es dataset_dir/annots/image_id.xml 
        path = info['annotation']
        
         # se llama al método extract_boxes (arriba) para obtener bndbox del archivo .xml
        boxes, w, h = self.extract_boxes(path)
        
        # se crea una cantidad de len(boxes) de mascaras de alto 'h' y ancho 'w'
        masks = zeros([h, w, len(boxes)], dtype='uint8')

        class_ids = list()
        
        ## se recorren todos los boxes y generamos máscaras (máscara de bndbox) y class id para cada instancia
        # las máscaras tendrán forma rectangular ya que hemos usado bndboxes para etiquetas
        # por ejemplo: si 2.jpg tiene tres objetos, tendremos las siguientes máscaras y class_ids.
        
        # 000000000 000000000 000003330 111100000
        # 000011100 022200000 000003330 111100000
        # 000011100 022200000 000003330 111100000
        # 000000000 022200000 000000000 000000000
        #    1         2          3         1<- class_ids
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            
            # box[4] will have the name of the class for a particular damage
            if (box[4] == 'arandano'):
                masks[row_s:row_e, col_s:col_e, i] = 1
                class_ids.append(self.class_names.index('arandano'))
            elif(box[4] == 'arandano-maduro'):
                masks[row_s:row_e, col_s:col_e, i] = 2
                class_ids.append(self.class_names.index('arandano-maduro')) 
                
        # retorna mascaras y class_ids como arreglo
        return masks, asarray(class_ids, dtype='int32')
    
    # esta funciones toma el image_id y retorna la ruta de la imagen 
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

In [5]:
# damage configuration class, you can change values of hyper parameters here
class ConfigArandanos(Config):

    # nombre de la configuracion
    NAME = "arandano_cfg_test_5_fase_2_2_"    
    
# clase arandano + clase background 
    NUM_CLASSES = 1 + 2
    
    # pasos por epoch y confianza minima    
    # STEPS_PER_EPOCH = cantidad de lotes/batchs
    #STEPS_PER_EPOCH = 73   # por epoch se entrenaran 73 lotes de 4 imagenes, dataset = 292
    STEPS_PER_EPOCH = 146
    
    # tasa de aprendizaje y momentum
    LEARNING_RATE=0.001
    LEARNING_MOMENTUM = 0.8
    
    # penalización de regularización
    WEIGHT_DECAY = 0.0001
    
    # el tamaño de la imagen está controlado por este parámetro
    IMAGE_MIN_DIM = 512
    
    # pasos de validación
    VALIDATION_STEPS = 50
    
    # número de regiones de interés generadas por imagen
    Train_ROIs_Per_Image = 200
    
    # escala de anclas RPN y proporciones (ratios) para encontrar la ROI
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)    # Longitud del lado del ancla cuadrada, en píxeles 
    RPN_ANCHOR_RATIOS = [0.5, 1, 1.5]   # Proporciones de anclas por cada celda (ancho/alto). Un valor de 1 representa un ancla cuadrada y 0,5 es un ancla ancha 
    
    MINI_MASK_SHAPE = (56, 56)
    
    #DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0    
    DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

    IMAGES_PER_GPU = 2
    
    
ConfigArandanos().display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
DEVICE                         /gpu:0
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.8
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE 

### Entrenamiento

In [6]:
cd /tf/PT_JoseVeloso/Mask_RCNN-master_matterport/

/tf/PT_JoseVeloso/Mask_RCNN-master_matterport


In [7]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

log_dir = "./logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq=1)



In [8]:
#pesos = 'mask_rcnn_coco.h5'
#pesos = 'mask_rcnn_damage_cfg_0049.h5'
#pesos = 'arandano_cfg_test_5_20220907T0914/mask_rcnn_arandano_cfg_0300.h5'
#pesos = 'arandano_cfg_test_5_20220907T0914/mask_rcnn_arandano_cfg_0300.h5'
pesos = 'arandano_cfg_test_6_2_20220918T1934/mask_rcnn_arandano_cfg_6_2__0100.h5'

conjunto_datos = 'customImages/test_5_fase_2'


In [9]:
# cargar dataset de entrenamiento
train_set = DatasetArandanos()
train_set.load_dataset(conjunto_datos, is_train=True)
train_set.prepare()

# cargar dataset de test 
test_set = DatasetArandanos()
test_set.load_dataset(conjunto_datos, is_train=False)
test_set.prepare()

# preparar la configuración llamando a la clase de configuración definida por el usuario
config = ConfigArandanos()

# definir el modelo
with tf.device(config.DEVICE):
    model = MaskRCNN(mode='training', model_dir='./', config=config)

# cargar pesos del modelo 
weights_path = pesos

# cargar los pesos del modelo
model.load_weights(weights_path, 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

# iniciar entrenamiento del modelo
#model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, custom_callbacks = [tensorboard_callback], epochs=300, layers='all')
    
#en honor al tiempo se realiza con 150 epoch
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, 
            custom_callbacks = [tensorboard_callback], epochs=120, layers='all')



Starting at epoch 0. LR=0.001

Checkpoint Path: ./arandano_cfg_test_5_fase_2_2_20220919T0845/mask_rcnn_arandano_cfg_test_5_fase_2_2__{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b       

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/150


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/ROI/GatherV2_2_grad/Reshape_1:0", shape=(6000,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/ROI/GatherV2_2_grad/Reshape:0", shape=(6000, 4), dtype=float32), dense_shape=Tensor("training/SGD/gradients/gradients/ROI/GatherV2_2_grad/Cast:0", shape=(2,), dtype=int32, device=/device:GPU:0))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/ROI/GatherV2_3_grad/Reshape_1:0", shape=(6000,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/ROI/GatherV2_3_grad/Reshape:0", shape=(6000, 4), dtype=float32), dense_shape=Tensor("train

146/146 [==============================] - ETA: 0s - batch: 72.5000 - size: 2.0000 - loss: 4.1918 - rpn_class_loss: 0.8899 - rpn_bbox_loss: 1.5781 - mrcnn_class_loss: 0.3421 - mrcnn_bbox_loss: 0.8632 - mrcnn_mask_loss: 0.5185

...

Epoch 61/150
146/146 [==============================] - 643s 4s/step - batch: 72.5000 - size: 2.0000 - loss: 0.5424 - rpn_class_loss: 0.0248 - rpn_bbox_loss: 0.0721 - mrcnn_class_loss: 0.1201 - mrcnn_bbox_loss: 0.0876 - mrcnn_mask_loss: 0.2379 - val_loss: 2.8901 - val_rpn_class_loss: 0.9027 - val_rpn_bbox_loss: 0.5803 - val_mrcnn_class_loss: 0.7412 - val_mrcnn_bbox_loss: 0.3147 - val_mrcnn_mask_loss: 0.3512
Epoch 62/150
146/146 [==============================] - 689s 5s/step - batch: 72.5000 - size: 2.0000 - loss: 0.5404 - rpn_class_loss: 0.0265 - rpn_bbox_loss: 0.0691 - mrcnn_class_loss: 0.1196 - mrcnn_bbox_loss: 0.0879 - mrcnn_mask_loss: 0.2373 - val_loss: 3.1989 - val_rpn_class_loss: 1.1785 - val_rpn_bbox_loss: 0.5454 - val_mrcnn_class_loss: 0.8101 - val_mrcnn_bbox_loss: 0.3157 - val_mrcnn_mask_loss: 0.3492
Epoch 63/150
146/146 [==============================] - 701s 5s/step - batch: 72.5000 - size: 2.0000 - loss: 0.5130 - rpn_class_loss: 0.0216 - rpn_bbox_loss: 0.0652 - mrcnn_class

Epoch 81/150
146/146 [==============================] - 819s 6s/step - batch: 72.5000 - size: 2.0000 - loss: 0.4291 - rpn_class_loss: 0.0112 - rpn_bbox_loss: 0.0521 - mrcnn_class_loss: 0.0893 - mrcnn_bbox_loss: 0.0680 - mrcnn_mask_loss: 0.2086 - val_loss: 3.7945 - val_rpn_class_loss: 1.6652 - val_rpn_bbox_loss: 0.6778 - val_mrcnn_class_loss: 0.8160 - val_mrcnn_bbox_loss: 0.2800 - val_mrcnn_mask_loss: 0.3554
Epoch 82/150
146/146 [==============================] - 634s 4s/step - batch: 72.5000 - size: 2.0000 - loss: 0.4350 - rpn_class_loss: 0.0116 - rpn_bbox_loss: 0.0557 - mrcnn_class_loss: 0.0919 - mrcnn_bbox_loss: 0.0681 - mrcnn_mask_loss: 0.2076 - val_loss: 3.3588 - val_rpn_class_loss: 1.3054 - val_rpn_bbox_loss: 0.5903 - val_mrcnn_class_loss: 0.7559 - val_mrcnn_bbox_loss: 0.2960 - val_mrcnn_mask_loss: 0.4112
Epoch 83/150
146/146 [==============================] - 655s 5s/step - batch: 72.5000 - size: 2.0000 - loss: 0.4288 - rpn_class_loss: 0.0122 - rpn_bbox_loss: 0.0545 - mrcnn_class

Epoch 101/150
146/146 [==============================] - 709s 5s/step - batch: 72.5000 - size: 2.0000 - loss: 0.3404 - rpn_class_loss: 0.0069 - rpn_bbox_loss: 0.0386 - mrcnn_class_loss: 0.0672 - mrcnn_bbox_loss: 0.0496 - mrcnn_mask_loss: 0.1781 - val_loss: 4.3700 - val_rpn_class_loss: 2.0680 - val_rpn_bbox_loss: 0.8351 - val_mrcnn_class_loss: 0.8013 - val_mrcnn_bbox_loss: 0.2824 - val_mrcnn_mask_loss: 0.3831
Epoch 102/150
146/146 [==============================] - 789s 5s/step - batch: 72.5000 - size: 2.0000 - loss: 0.3505 - rpn_class_loss: 0.0067 - rpn_bbox_loss: 0.0416 - mrcnn_class_loss: 0.0710 - mrcnn_bbox_loss: 0.0514 - mrcnn_mask_loss: 0.1799 - val_loss: 4.3842 - val_rpn_class_loss: 2.0300 - val_rpn_bbox_loss: 0.8034 - val_mrcnn_class_loss: 0.8737 - val_mrcnn_bbox_loss: 0.2922 - val_mrcnn_mask_loss: 0.3850
Epoch 103/150
146/146 [==============================] - 605s 4s/step - batch: 72.5000 - size: 2.0000 - loss: 0.3478 - rpn_class_loss: 0.0065 - rpn_bbox_loss: 0.0393 - mrcnn_cl

Epoch 121/150
146/146 [==============================] - 693s 5s/step - batch: 72.5000 - size: 2.0000 - loss: 0.2716 - rpn_class_loss: 0.0037 - rpn_bbox_loss: 0.0272 - mrcnn_class_loss: 0.0522 - mrcnn_bbox_loss: 0.0365 - mrcnn_mask_loss: 0.1519 - val_loss: 5.0897 - val_rpn_class_loss: 2.5990 - val_rpn_bbox_loss: 0.8263 - val_mrcnn_class_loss: 0.9795 - val_mrcnn_bbox_loss: 0.2803 - val_mrcnn_mask_loss: 0.4046
Epoch 122/150
 90/146 [=================>............] - ETA: 1:36 - batch: 44.5000 - size: 2.0000 - loss: 0.3035 - rpn_class_loss: 0.0045 - rpn_bbox_loss: 0.0343 - mrcnn_class_loss: 0.0602 - mrcnn_bbox_loss: 0.0411 - mrcnn_mask_loss: 0.1633

KeyboardInterrupt: 

In [10]:
%tensorboard --logdir logs/fit